# **Classification Challenge**

`Tópicos Especiais em Computação VIII`

Using Random Forests and Multi Layer Perceptrons to predict hospital readmissions of diabetic patients

*Luiz Henrique Rigo Faccio*

## **Importing Libraries and loading dataset**

In [87]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA


In [2]:
# Loading informations
folder = "diabetes"
diabetes = pd.read_csv(f'{folder}/diabetic_data.csv')
mapping = pd.read_csv(f'{folder}/IDS_mapping.csv')
admission_type_mapping = mapping[0:7]
discharge_disposition_mapping = mapping[10:40].reset_index(drop=True)
admission_source_mapping = mapping[42:67].reset_index(drop=True)

## **Observing the dataset**

In [3]:
print(diabetes.shape)
print(diabetes.info())
print(diabetes.describe(include='all'))

print(diabetes["readmitted"].value_counts())

print()
for c in diabetes.columns:
    print(f"Column {c}", end="\n\t\t")
    print(diabetes[c].unique(), end="\n\n")

(101766, 50)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int

## **Data treatment**

The `pre_process_new_diabetes_data()` function can be used to process new information so that it can be used in the model

In [ ]:
def pre_process_diabetes_data(data):
    """
        Args:
            df (pandas DataFrame): Dataset to be processed

        Returns:
            df (pandas DataFrame): Processed dataset
            features (list): List of features used in the model

        This function processes diabetes data from the original form to the training-ready form, to be used to train the models.
    """    

    # Dropping IDs and unnecessary columns and standadizing the missing values

    df = data.copy()

    df.drop(columns=['encounter_id', 'patient_nbr', 'payer_code'], inplace=True)
    df.replace(["?", 'Unknown/Invalid'], pd.NA, inplace=True)

    # Also drppping columns with too little information (Mostly null values)
    
    df.drop(columns=['weight', 'medical_specialty', 'max_glu_serum', 'A1Cresult'], inplace=True)

    # Joining tables

    df = df.join(admission_type_mapping["description"], how='left', on='admission_type_id').rename(columns={"description": "admission_type"}).drop(columns=['admission_type_id'])
    df = df.join(discharge_disposition_mapping["description"], how='left', on='discharge_disposition_id').rename(columns={"description": "discharge_disposition"}).drop(columns=['discharge_disposition_id'])
    df = df.join(admission_source_mapping["description"], how='left', on='admission_source_id').rename(columns={"description": "admission_source"}).drop(columns=['admission_source_id'])

    # Dropping rows with missing values

    df.dropna(axis=0, how='any', inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Treating the target 

    y = df["readmitted"].replace({'NO': 0, '>30': 1, '<30': 2})
    df.drop(columns=['readmitted'], inplace=True)

    # Scaling numerical variables

    numerical_cols =df.select_dtypes(include=['int64', 'float64']).columns
    scaler = StandardScaler()
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

    # Getting dummies for categorical variables 

    categorical_columns = df.select_dtypes(include=['object']).columns

    temp = pd.DataFrame()
    for column in categorical_columns:
        dummies = pd.get_dummies(df[column], prefix=column, dtype=int)
        temp = pd.concat([temp, dummies], axis=1)
        df.drop(columns=[column], inplace=True)


    # Removing columns with low variance
    
    selector = VarianceThreshold(threshold=0.01)    ## Remove collumns with variance lower than 0.01, that is, 
                                                    ## those collumns have the same value for 99.9% of the rows
                                                    
    temp = pd.DataFrame(selector.fit_transform(temp), columns=temp.columns[selector.get_support()])

    final = pd.concat([df, temp, y], axis=1)
    return final, final.columns.tolist()

In [49]:
def pre_process_new_diabetes_data(data, features):
    """
        Args:
            df (pandas DataFrame): Dataset to be processed
            features (list): List of features used in the model

        Returns:
            df (pandas DataFrame): Processed dataset

        This function processes diabetes data from the original form to the model-ready form.
    """    

    # Dropping IDs and unnecessary columns and standadizing the missing values

    df = data.copy()

    df.drop(columns=['encounter_id', 'patient_nbr', 'payer_code'], inplace=True)
    df.replace(["?", 'Unknown/Invalid'], pd.NA, inplace=True)

    # Also drppping columns with too little information (Mostly null values)
    
    df.drop(columns=['weight', 'medical_specialty', 'max_glu_serum', 'A1Cresult'], inplace=True)

    # Joining tables

    df = df.join(admission_type_mapping["description"], how='left', on='admission_type_id').rename(columns={"description": "admission_type"}).drop(columns=['admission_type_id'])
    df = df.join(discharge_disposition_mapping["description"], how='left', on='discharge_disposition_id').rename(columns={"description": "discharge_disposition"}).drop(columns=['discharge_disposition_id'])
    df = df.join(admission_source_mapping["description"], how='left', on='admission_source_id').rename(columns={"description": "admission_source"}).drop(columns=['admission_source_id'])

    # Dropping rows with missing values

    df.dropna(axis=0, how='any', inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Treating the target 

    y = df["readmitted"].replace({'NO': 0, '>30': 1, '<30': 2})
    df.drop(columns=['readmitted'], inplace=True)

    # Scaling numerical variables

    numerical_cols =df.select_dtypes(include=['int64', 'float64']).columns
    scaler = StandardScaler()
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

    # Getting dummies for categorical variables 

    categorical_columns = df.select_dtypes(include=['object']).columns

    temp = pd.DataFrame()
    for column in categorical_columns:
        dummies = pd.get_dummies(df[column], prefix=column, dtype=int)
        temp = pd.concat([temp, dummies], axis=1)
        df.drop(columns=[column], inplace=True)
    
    

    final = pd.concat([df, temp, y], axis=1)

    current_cols = final.columns.tolist()
    missing_cols = set(features) - (set(current_cols))

    for col in missing_cols:
        final[col] = 0

    return final[features]

In [98]:
# Processing the dataset

diabetes_processed, features = pre_process_diabetes_data(diabetes)
diabetes_processed.info(verbose=True, memory_usage=True)
diabetes_processed.head(7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93054 entries, 0 to 93053
Data columns (total 358 columns):
 #    Column              Dtype  
---   ------              -----  
 0    time_in_hospital    float64
 1    num_lab_procedures  float64
 2    num_procedures      float64
 3    num_medications     float64
 4    number_outpatient   float64
 5    number_emergency    float64
 6    number_inpatient    float64
 7    number_diagnoses    float64
 8    pca_1               float64
 9    pca_2               float64
 10   pca_3               float64
 11   pca_4               float64
 12   pca_5               float64
 13   pca_6               float64
 14   pca_7               float64
 15   pca_8               float64
 16   pca_9               float64
 17   pca_10              float64
 18   pca_11              float64
 19   pca_12              float64
 20   pca_13              float64
 21   pca_14              float64
 22   pca_15              float64
 23   pca_16              float64
 24   

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,pca_21,pca_22,pca_23,pca_24,pca_25,pca_26,pca_27,pca_28,pca_29,pca_30,pca_31,pca_32,pca_33,pca_34,pca_35,pca_36,pca_37,pca_38,pca_39,pca_40,pca_41,pca_42,pca_43,pca_44,pca_45,pca_46,pca_47,pca_48,pca_49,pca_50,pca_51,pca_52,pca_53,pca_54,pca_55,pca_56,pca_57,pca_58,pca_59,pca_60,pca_61,pca_62,pca_63,pca_64,pca_65,pca_66,pca_67,pca_68,pca_69,pca_70,pca_71,pca_72,pca_73,pca_74,pca_75,pca_76,pca_77,pca_78,pca_79,pca_80,pca_81,pca_82,pca_83,pca_84,pca_85,pca_86,pca_87,pca_88,pca_89,pca_90,pca_91,pca_92,pca_93,pca_94,pca_95,pca_96,pca_97,pca_98,pca_99,pca_100,pca_101,pca_102,pca_103,pca_104,pca_105,pca_106,pca_107,pca_108,pca_109,pca_110,pca_111,pca_112,pca_113,pca_114,pca_115,pca_116,pca_117,pca_118,pca_119,pca_120,pca_121,pca_122,pca_123,pca_124,pca_125,pca_126,pca_127,pca_128,pca_129,pca_130,pca_131,pca_132,pca_133,pca_134,pca_135,pca_136,pca_137,pca_138,pca_139,pca_140,pca_141,pca_142,pca_143,pca_144,pca_145,pca_146,pca_147,pca_148,pca_149,pca_150,pca_151,pca_152,pca_153,pca_154,pca_155,pca_156,pca_157,pca_158,pca_159,pca_160,pca_161,pca_162,pca_163,pca_164,pca_165,pca_166,pca_167,pca_168,pca_169,pca_170,pca_171,pca_172,pca_173,pca_174,pca_175,pca_176,pca_177,pca_178,pca_179,pca_180,pca_181,pca_182,pca_183,pca_184,pca_185,pca_186,pca_187,pca_188,pca_189,pca_190,pca_191,pca_192,pca_193,pca_194,pca_195,pca_196,pca_197,pca_198,pca_199,pca_200,pca_201,pca_202,pca_203,pca_204,pca_205,pca_206,pca_207,pca_208,pca_209,pca_210,pca_211,pca_212,pca_213,pca_214,pca_215,pca_216,pca_217,pca_218,pca_219,pca_220,pca_221,pca_222,pca_223,pca_224,pca_225,pca_226,pca_227,pca_228,pca_229,pca_230,pca_231,pca_232,pca_233,pca_234,pca_235,pca_236,pca_237,pca_238,pca_239,pca_240,pca_241,pca_242,pca_243,pca_244,pca_245,pca_246,pca_247,pca_248,pca_249,pca_250,pca_251,pca_252,pca_253,pca_254,pca_255,pca_256,pca_257,pca_258,pca_259,pca_260,pca_261,pca_262,pca_263,pca_264,pca_265,pca_266,pca_267,pca_268,pca_269,pca_270,pca_271,pca_272,pca_273,pca_274,pca_275,pca_276,pca_277,pca_278,pca_279,pca_280,pca_281,pca_282,pca_283,pca_284,pca_285,pca_286,pca_287,pca_288,pca_289,pca_290,pca_291,pca_292,pca_293,pca_294,pca_295,pca_296,pca_297,pca_298,pca_299,pca_300,pca_301,pca_302,pca_303,pca_304,pca_305,pca_306,pca_307,pca_308,pca_309,pca_310,pca_311,pca_312,pca_313,pca_314,pca_315,pca_316,pca_317,pca_318,pca_319,pca_320,pca_321,pca_322,pca_323,pca_324,pca_325,pca_326,pca_327,pca_328,pca_329,pca_330,pca_331,pca_332,pca_333,pca_334,pca_335,pca_336,pca_337,pca_338,pca_339,pca_340,pca_341,pca_342,pca_343,pca_344,pca_345,pca_346,pca_347,pca_348,pca_349,readmitted
0,-0.482483,0.773285,-0.794457,0.230064,-0.279855,-0.209539,-0.511512,0.808580,0.769268,-0.757539,0.497789,0.057625,0.040615,-0.057489,-1.000206,0.101826,-0.189102,-0.158361,-0.098900,-0.012402,-0.064456,-0.047946,-0.239136,0.055188,-0.050543,-0.726064,0.348675,-0.090542,0.011158,-0.162231,-0.058811,0.098186,0.012489,0.009834,0.039220,-0.051286,-0.168501,-0.032248,-0.018271,-0.094385,0.172524,-0.087761,-0.114393,-0.042958,0.025713,0.049369,-0.067771,-0.034557,0.007795,0.025082,-0.036497,0.027690,-0.002985,-0.019629,0.024106,0.140470,-0.026791,-0.139260,-0.040722,0.034904,-0.072724,-0.010634,-0.039741,-0.229648,-0.067005,0.002648,-0.012962,0.083386,-0.477854,-0.244892,0.369777,-0.218192,0.332493,0.344090,0.091328,0.221074,-0.270087,0.245735,-0.019882,-0.011109,0.068602,0.151665,-0.050074,0.022724,0.107045,-0.357192,-0.114613,0.054730,-0.302441,0.470706,-0.373551,0.028134,-0.297156,0.270441,-0.161224,-0.116116,-0.102770,-0.106794,-0.142817,-0.086037,-0.146141,-0.064041,-0.158779,0.073206,0.024134,-0.085089,-0.021692,0.031956,-0.056387,0.024949,0.003167,-0.021594,-0.003720,0.000945,0.010032,-0.027723,-0.018157,0.006389,-0.024530,0.021955,0.030553,-0.004

## **The model** 

Two different models were used: RandomForest and Multi Layer Perceptron Classifier (RNN)

A Grid Search was executed to find the best combination of parameters for this problem, to each of the two models

### **Finding out the best parameters**

A grid Search performs a series of model fits using many different pre-defined parameters and makes it easy to compare wich are the best

In [6]:
def best_model_Random_Forest(processed_data: pd.DataFrame, target: str, param_grid: dict):
    """
        Args:
            processed_data (pandas DataFrame): Processed diabetes dataset to be used for training
            target (str): Target variable name
            param_grid (dict): Dictionary containing hyperparameters for Random Forest model

        Returns:
            grid_search, best_model (Grid_Search, RandomForestClassifier): Grid Search results and best Random Forest model after hyperparameter tuning

        This function performs hyperparameter tuning for a Random Forest model using GridSearchCV on the processed diabetes dataset.
    """

    # Splitting Data
    X = processed_data.drop(columns=[target])
    y = processed_data[target]
    
    scoring = 'accuracy'
    
    grid_search = GridSearchCV(RandomForestClassifier(verbose=0), 
                               param_grid, 
                               cv=3, 
                               scoring=scoring, 
                               n_jobs=-1, 
                               verbose=1)
    grid_search.fit(X, y)

    # Visualizing the grid search results
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    print("Score type: ", scoring)
    display(pd.concat([cv_results_df[f'rank_test_score'], 
                        cv_results_df[f'mean_test_score'], 
                        cv_results_df[f'std_test_score'],
                        cv_results_df[('param_' + str(k) for k in param_grid.keys())]], 
                    axis=1).sort_values(by='rank_test_score'))
    
    return grid_search, grid_search.best_estimator_


In [7]:
def best_model_MLPClassifier(processed_data: pd.DataFrame, target: str, param_grid: dict):
    """
        Args:
            processed_data (pandas DataFrame): Processed diabetes dataset to be used for training
            target (str): Target variable name
            param_grid (dict): Dictionary containing hyperparameters for MLPClassifier model

        Returns:
            grid_search, best_model (Grid_Search, MLPClassifier): Grid Serach results and best MLPClassifier model after hyperparameter tuning

        This function performs hyperparameter tuning for a MLPCLassifier model using GridSearchCV on the processed diabetes dataset.
    """

    # Splitting Data
    X = processed_data.drop(columns=[target])
    y = processed_data[target]
    
    scoring= ['accuracy','f1_macro']

    grid_search = GridSearchCV(MLPClassifier(max_iter=250, 
                                             verbose=False, 
                                             early_stopping=True, 
                                             n_iter_no_change=30), 
                               param_grid, 
                               cv=3, 
                               scoring=scoring, 
                               n_jobs=-1,
                               refit=scoring[0],
                               verbose=1)
    grid_search.fit(X, y)

    # Visualizing the grid search results
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    if len(scoring) == 1:
        columns = ['rank_test_score', 'mean_test_score', 'std_test_score']
        sort = 'rank_test_score'
    else:
        columns = [f'rank_test_{s}' for s in scoring] + [f'mean_test_{s}' for s in scoring] + [f'std_test_{s}' for s in scoring]
        sort = [f'rank_test_{s}' for s in scoring]
    
    print("Score type: ", scoring)
    display(pd.concat([
         cv_results_df[columns], 
         cv_results_df[[('param_' + str(k)) for k in param_grid.keys()]]], axis=1)
        .sort_values(by=sort))
    
    return grid_search, grid_search.best_estimator_


#### DO NOT RUN THE NEXT TWO CELLS

They might take hours to process

This process has already been done and the best parameters are in the next section

In [ ]:
## Fiding ou the best parameters for the Random Forest model (Abou 42 Min of processing time for 192 fits)

# Defining the parameters to be combined and tested for Random Forest
param_grid_rf = {
    'n_estimators': [200, 500],             
    'max_depth': [20, 50],             
    'min_samples_split': [5, 10],               
    'max_features': ['sqrt', 'log2'],      
    'bootstrap': [True],                  
    'criterion': ['gini', 'entropy'],             
    'class_weight': ['balanced', 'balanced_subsample']  # To handle unbalanced target classes
}

grid_rf, m_rf = best_model_Random_Forest(diabetes_processed, "readmitted", param_grid_rf)

Fitting 3 folds for each of 64 candidates, totalling 192 fits
Score type:  accuracy


,rank_test_score,mean_test_score,std_test_score,param_n_estimators,param_max_depth,param_min_samples_split,param_max_features,param_bootstrap,param_criterion,param_class_weight
57,1,0.566983,0.005813,500,50,5,sqrt,True,entropy,balanced_subsample
25,2,0.566456,0.005751,500,50,5,sqrt,True,entropy,balanced
61,3,0.565467,0.004747,500,50,5,log2,True,entropy,balanced_subsample
29,4,0.565209,0.004830,500,50,5,log2,True,entropy,balanced
45,5,0.565102,0.005554,500,50,5,log2,True,gini,balanced_subsample
...,...,...,...,...,...,...,...,...,...,...
38,60,0.527597,0.026985,200,20,10,log2,True,gini,balanced_subsample
6,61,0.526855,0.027977,200,20,10,log2,True,gini,balanced
55,62,0.526673,0.029115,500,20,10,log2,True,entropy,balanced_subsample
22,63,0.525931,0.027957,200,20,10,log2,True,entropy,balanced


In [153]:
# Finding out the best parameters for the MLPClassifier model (Aboutv 7 Min of processing time)

param_grid_mlp = {
    'hidden_layer_sizes': [(128,), (128, 64), (256, 128)],
    'activation': ['relu', 'tanh'],
    'alpha': [1e-3, 1e-2],
    'learning_rate_init': [0.001, 0.005],
    'solver': ['adam', 'sgd']
}

grid_mlp, m_mlp = best_model_MLPClassifier(diabetes_processed, "readmitted", param_grid_mlp)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
Score type:  ['accuracy', 'f1_macro']


,rank_test_accuracy,rank_test_f1_macro,mean_test_accuracy,mean_test_f1_macro,std_test_accuracy,std_test_f1_macro,param_hidden_layer_sizes,param_activation,param_alpha,param_learning_rate_init,param_solver
45,1,18,0.571528,0.382580,0.005910,0.019751,"(256, 128)",tanh,0.010,0.001,sgd
14,2,43,0.571002,0.376697,0.007059,0.021310,"(128,)",relu,0.010,0.005,adam
26,3,5,0.570411,0.389400,0.004243,0.018981,"(128,)",tanh,0.001,0.005,adam
4,4,13,0.570067,0.384810,0.006295,0.015040,"(128, 64)",relu,0.001,0.001,adam
24,5,4,0.570035,0.390390,0.004888,0.011951,"(128,)",tanh,0.001,0.001,adam
22,6,1,0.570024,0.391536,0.008124,0.022262,"(256, 128)",relu,0.010,0.005,adam
5,7,39,0.570013,0.378444,0.005719,0.019432,"(128, 64)",relu,0.001,0.001,sgd
25,8,30,0.569658,0.380457,0.007594,0.025704,"(128,)",tanh,0.001,0.001,sgd
21,9,41,0.569411,0.377912,0.005251,0.018311,"(256, 128)",relu,0.010,0.001,sgd
43,10,8,0.569089,0.387051,0.005086,0.016675,"(128, 64)",tanh,0.010,0.005,sgd


### **Training the final models with the defined parameters**

**Best parameters found:**

rf_best_params: `{'bootstrap': True, 'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'max_depth': 50, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 500}`

mlp_best_params: `{'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (256, 128), 'learning_rate_init': 0.001, 'solver': 'sgd'}`


In [82]:
def single_model_RNN(processed_data: pd.DataFrame, target: str, params: dict):
    """
        Args:
            processed_data (pandas DataFrame): Processed diabetes dataset to be used for training
            target (str): Target variable name
            params (dict): Dictionary containing hyperparameters for MLPClassifier model

        Returns:
            model (MLPClassifier): Trained MLP model
            X_test (pandas DataFrame): Test data to be used for evaluation
            y_test (pandas Series): Test data to be used for evaluation

        This function trains a Multi-Layer Perceptron (MLP) model on the processed diabetes dataset, using predefined hyperparameters.
    """    

    # Splitting the data into train and test sets
    X = processed_data.drop(columns=[target])
    y = processed_data[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = MLPClassifier(**params, early_stopping=True,
                                    max_iter=250, 
                                    n_iter_no_change=30)

    model.fit(X_train, y_train)

    return model, X_test, y_test


In [10]:
def single_model_Random_Forest(processed_data: pd.DataFrame, target: str, params: dict):
    """
        Args:
            processed_data (pandas DataFrame): Processed diabetes dataset to be used for training
            target (str): Target variable name
            params (dict): Dictionary containing hyperparameters for Random Forest model

        Returns:
            rf (RandomForestClassifier): Trained Random Forest model
            X_test (pandas DataFrame): Test data to be used for evaluation
            y_test (pandas Series): Test data to be used for evaluation

        This function trains a Random Forest model on the processed diabetes dataset, using predefined hyperparameters.
    """    

    # Splitting the data into train and test sets
    X = processed_data.drop(columns=[target])
    y = processed_data[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Creating and training the Random Forest model
    rf = RandomForestClassifier(**params, n_jobs=-1, verbose=0)
    
    rf.fit(X_train, y_train)

    return rf, X_test, y_test


In [99]:
rf, rf_X_test, rf_Y_test = single_model_Random_Forest(diabetes_processed, "readmitted", {'bootstrap': True, 'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'max_depth': 50, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 2000})

## rf_best_params: {'bootstrap': True, 'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'max_depth': 50, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 500}

mlp, mlp_X_test, mlp_y_test = single_model_RNN(diabetes_processed, "readmitted", {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (256, 128), 'learning_rate': 'adaptive', 'solver': 'sgd'})

## mlp_best_params: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (256, 128), 'learning_rate_init': 0.001, 'solver': 'sgd'}

C:\Users\Pichau\AppData\Roaming\Python\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


## **Evaluating the models**

In [100]:
print("------------------RANDOM FOREST------------------", end="\n\n")
rf_predictions = rf.predict(rf_X_test)

print("Accuracy:", accuracy_score(rf_Y_test, rf_predictions))
print("\nClassification Report:\n", classification_report(rf_Y_test, rf_predictions))

print("Confusion Matrix:")
display(pd.crosstab(rf_Y_test, rf_predictions, rownames=['Actual'], colnames=['Predicted'], margins=True))

print("Feature Importance:")
display(pd.DataFrame(rf.feature_importances_, index=rf_X_test.columns, columns=['Importance']).sort_values(by='Importance', ascending=False).head(10))

print("---------------MULTI LAYER PERCEPTRON---------------", end="\n\n")
mlp_predictions = mlp.predict(mlp_X_test)

print("Accuracy:", accuracy_score(mlp_y_test, mlp_predictions))
print("\nClassification Report:\n", classification_report(mlp_y_test, mlp_predictions))

print("Confusion Matrix:")
display(pd.crosstab(mlp_y_test, mlp_predictions, rownames=['Actual'], colnames=['Predicted'], margins=True))


------------------RANDOM FOREST------------------

Accuracy: 0.5648809843640857

Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.91      0.70      9838
           1       0.51      0.24      0.33      6572
           2       0.53      0.01      0.03      2201

    accuracy                           0.56     18611
   macro avg       0.54      0.39      0.35     18611
weighted avg       0.55      0.56      0.49     18611

Confusion Matrix:


Predicted,0,1,2,All
Actual,,,,
0,8910,915,13,9838
1,4985,1572,15,6572
2,1594,576,31,2201
All,15489,3063,59,18611


Feature Importance:


,Importance
number_inpatient,0.013294
pca_5,0.004650
pca_46,0.004446
pca_30,0.004305
pca_28,0.003978
pca_16,0.003847
pca_15,0.003790
pca_6,0.003756
pca_54,0.003705
pca_2,0.003692


---------------MULTI LAYER PERCEPTRON---------------

Accuracy: 0.5783676320455644

Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.83      0.70      9838
           1       0.50      0.38      0.43      6572
           2       0.41      0.03      0.06      2201

    accuracy                           0.58     18611
   macro avg       0.51      0.42      0.40     18611
weighted avg       0.55      0.58      0.53     18611

Confusion Matrix:


Predicted,0,1,2,All
Actual,,,,
0,8177,1620,41,9838
1,3992,2511,69,6572
2,1259,866,76,2201
All,13428,4997,186,18611
